In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from glob import glob

In [4]:
import itertools

In [5]:
import pygsheets as pyg

In [6]:
import time

#### some pandas opts

In [7]:
pd.options.display.max_columns = 100

In [8]:
pd.options.display.max_rows = 500

#### authorize pyg and set retry limit

In [9]:
gc = pyg.authorize(outh_file="./client_secret_986841527205-ndf6fbsfmrhjinnofm47j6olvmm11smn.apps.googleusercontent.com.json",
                   outh_nonlocal=True,
                   retries=5)

#### title for the google sheet and xls

In [10]:
sheet_name = "W303_nup133_day42"

#### xls output directory

In [11]:
xls_out_dir = "/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-nup133-day42/"

#### where are the files?

In [12]:
%ls /home/dizak/Pulpit/BIONAS/G148/SNPs_calling/

16 documents from consensus.geneious*
16 documents from WT-day70 to WT-zero point.geneious*
BY-day0/
BY-nup133-day70/
BY-WT-day70/
C2_merged_dfs.csv*
consensus/
mapped_contigs/
mapped_reads/
nup mapped to wt cons.geneious*
W303-cog7-day42/
W303-nup133-day42/
WT-day70/
WT-day70.xlsx*


In [13]:
sampling_levels = ["C1", "C2", "C3"]

In [14]:
strain_gene_day_N1_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-nup133-day42/N1/*csv")

In [15]:
strain_gene_day_N2_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-nup133-day42/N2/*csv")

In [16]:
strain_gene_day_N3_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-nup133-day42/N3/*csv")

#### get non-redundant list of genes in the inputfiles

In [17]:
def find_flat_value(inputfiles_list,
                    col_name = "CDS"):
    """
    Get flat list of desired values from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    col_name: str
        Desired column name in the input CSV file.
    
    Returns
    -------
    list of desired values.
    """
    values_list = []
    for i in inputfiles_list:
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        elif col_name not in df.columns:
            pass
        else:
            values_list.append(df[col_name].dropna().drop_duplicates().tolist())
    return list(itertools.chain.from_iterable(values_list))

#### get values from input files by key

In [18]:
def find_by_key(inputfiles_list,
                key):
    """
    Get pandas.DataFrame selected by a given key from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    key: str, int, float, bool
        Key used as query against rows in the CSV files.
    value_col: str
        Column name which holds values to be returned.
    
    Returns
    -------
    dict of lists of desired values if pandas.Dataframe not empty
    None if pandas.DataFrame empty
    """
    values_list = []
    for i in inputfiles_list:
        filename = "".join(i.split("/")[-1].split(".")[:-1])
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        else:
            if isinstance(key, str) == True:
                df_dtype_sel = df.select_dtypes(include=["object"])
            elif isinstance(key, int) == True:
                df_dtype_sel = df.select_dtypes(include=["int"])
            elif isinstance(key, float) == True:
                df_dtype_sel = df.select_dtypes(include=["float"])
            elif isinstance(key, bool) == True:
                df_dtype_sel = df.select_dtypes(include=["bool"])
            else:
                raise ValueError("key must str, int, float or bool dtype")
            for col in df_dtype_sel.columns:
                df_sel = df[df_dtype_sel[col] == key]
                if len(df_sel) > 0:
                    return {"dataframe": df_sel,
                            "filename": filename}

#### get whole set of pandas.DataFrames selections in one dict

In [19]:
def get_dfs_set(key_list,
                files_list,
                vals=["Minimum",
                      "Maximum",
                      "Change"],
                df_key="dataframe",
                key_index_name="Gene",
                smpl_index_name="Sample",
                row_index_name="Number",
                index_by_key=True,
                smpl_index_val=None,):
    """
    Get desired values in pandas.Dataframe gathered in dict by
    the list of keys.
    
    Parameters
    -------
    key_list: list, tuple
        List of keys by which pandas.Dataframes are
        initially selected.
    files_list: list, tuple
        List of input CSV files.
    vals: list of str, default: ["Minimum", 
                                 "Maximum",
                                 "Change"]
        Columns names holding data in pandas.DataFrames
        selected.
    df_key: str, default: <"dataframe">
        Key for generic pandas.DataFrame selection for
        SNPs-sheets_merge.find_by_key function.
    key_index_name: str, default: <"Gene">
        Name for index of selection key.
    smpl_index_name: str, default: <"Sample">
        Name for index of sample.
    row_index_name: str, default: <"Number">
        Name for numeric row index.
    index_by_key: bool, default: True
        Enables multiindexing.
    smpl_index_val: str, default: <None>
        Adds sample name to multiindexing if not <None>
    """
    out_dict = {}
    for i in key_list:
        key_vals = find_by_key(files_list,
                               key=i)[df_key]
        out_dict[i] = key_vals[vals]
    if index_by_key is True:
        for i in out_dict:
            key_index = [i] * len(out_dict[i])
            if smpl_index_val is not None:
                smpl_index = [smpl_index_val] * len(out_dict[i])
                tpls = list(zip(*[key_index,
                                  smpl_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          smpl_index_name,
                                                          row_index_name])
            else:
                tpls = list(zip(*[key_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          row_index_name])
            out_dict[i].index = mindex
    return out_dict

#### merge any given number of dfs

In [20]:
def merge_dfs(dfs,
              sort_cols=["Minimum",
                         "Maximum"],
              reconstr_index=True):
    """
    Merge any number of pandas.DataFrame into one.
    Indexes must be identical in all the pandas.DataFrames.
    
    Parameters
    -------
    dfs: list
        list of pandas.DataFrames to merge.
    sort_cols: list, None
        list of col names to sort the final
        pandas.DataFrame by. No sorting if None.
    """
    for x in [set(i.index) for i in dfs]:
        assert len(x) == 1, "Indices are not homogenic."
    new_index = list(x)
    df = reduce(lambda df1, df2: pd.merge(left=df1,
                                          right=df2,
                                          how="outer"),
                dfs)
    if sort_cols is not None:
        df.sort(columns=sort_cols)
    if reconstr_index is True:
        df.index = len(df) * new_index
    return df

#### let's find out which CDS are present in all the files one by one

In [21]:
strain_gene_day_N1_CDSs = find_flat_value(strain_gene_day_N1_files)
strain_gene_day_N2_CDSs = find_flat_value(strain_gene_day_N2_files)
strain_gene_day_N3_CDSs = find_flat_value(strain_gene_day_N3_files)

#### let's gather some info about each of the CDS from each sample

In [22]:
strain_gene_day_N1_dfs = get_dfs_set(strain_gene_day_N1_CDSs,
                                     strain_gene_day_N1_files,
                                     smpl_index_val=sampling_levels[0])
strain_gene_day_N2_dfs = get_dfs_set(strain_gene_day_N2_CDSs,
                                     strain_gene_day_N2_files,
                                     smpl_index_val=sampling_levels[1])
strain_gene_day_N3_dfs = get_dfs_set(strain_gene_day_N3_CDSs,
                                     strain_gene_day_N3_files,
                                     smpl_index_val=sampling_levels[2])

#### let's unwind them all from this dict into lists

In [23]:
N1_dfs = [strain_gene_day_N1_dfs[i] for i in strain_gene_day_N1_dfs.keys()]
N2_dfs = [strain_gene_day_N2_dfs[i] for i in strain_gene_day_N2_dfs.keys()]
N3_dfs = [strain_gene_day_N3_dfs[i] for i in strain_gene_day_N3_dfs.keys()]

#### let's merge them to see changes between samples.
#### rememeber now cannot use the merge_dfs function since multindex is NOT homogenic

In [24]:
N1_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
                N1_dfs)

In [25]:
N2_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N2_dfs)

In [26]:
N3_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N3_dfs)

In [27]:
N1_N2_N3_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
                     [N1_df, N2_df, N3_df])

In [28]:
N1_N2_N3_df

Minimum  Maximum  \
Gene                       Sample Number                     
PFK27 CDS                  C1     7         66847    66847   
PIR3 CDS                   C1     167      144820   144820   
                                  168      144808   144808   
                                  169      144798   144798   
TDA1 CDS                   C1     24       853342   853342   
hypothetical protein CDS   C1     22       482031   482051   
                                  23       482026   482028   
                                  24       482019   482024   
                                  25       482017   482017   
                                  26       482009   482013   
                                  27       475807   475807   
                                  28       475798   475802   
                                  29       475792   475794   
                                  30       475785   475790   
                                  31       475783   475783   
                                  32       475765   475777   
                                  33       475757   475763   
                                  34       475749   475752   
                                  35       475747   475747   
                                  36       475743   475745   
                                  37       475740   475742   
                                  38       475738   475738   
                                  39       475734   475736   
                                  40       475730   475730   
                                  41       475725   475725   
                                  42       475721   475722   
                                  43       475717   475717   
ARG8 CDS                   C1     8         58930    58930   
                                  9         58927    58927   
                                  10        58924    58925   
PAU19 CDS                  C1     0        921918   921918   
                                  1        921909   921912   
                                  2        921906   921907   
                                  3        921903   921904   
                                  4        921895   921899   
                                  5        921889   921893   
                                  6        921887   921887   
                                  7        921880   921884   
                                  8        921876   921877   
                                  9        921871   921873   
                                  10       921868   921868   
                                  11       921865   921865   
                                  12       921860   921863   
FLO5 CDS                   C1     20       527089   527089   
                                  21       527026   527026   
                                  22       526993   526993   
                                  23       526988   526988   
                                  24       526979   526981   
                                  25       526888   526888   
                                  26       526852   526852   
                                  27       526849   526849   
                                  28       526816   526816   
YPS5 CDS                   C1     93         8654     8656   
                                  94         8650     8650   
CHL4 CDS                   C1     3        965718   965718   
MTL1 CDS                   C1     85       529985   529985   
HXT16 CDS                  C1     2        733951   733951   
                                  3        733820   733820   
                                  4        733817   733817   
                                  5        733802   733802   
                                  6        733792   733792   
                                  7        733218   733218   
GDS1 CDS                   C1     0       1004799  1004799   
FLO9 CDS    

#### let's make sheet on gdrive

In [29]:
sh = gc.create(sheet_name)

In [30]:
for i in set(strain_gene_day_N1_CDSs +
             strain_gene_day_N2_CDSs +
             strain_gene_day_N3_CDSs):
    time.sleep(3)
    wks = sh.add_worksheet(i)
    wks.set_dataframe(N1_N2_N3_df.xs(i),
                      (1, 1),
                      copy_index=True,
                      fit=True)

#### let's make xls

In [31]:
writer = pd.ExcelWriter("{}/{}.xls".format(xls_out_dir, sheet_name))

In [32]:
for i in set(strain_gene_day_N1_CDSs +
             strain_gene_day_N2_CDSs +
             strain_gene_day_N3_CDSs):
    N1_N2_N3_df.xs(i).to_excel(excel_writer=writer,
                               sheet_name=i)

In [33]:
writer.save()